In [2]:
import os
import sys
sys.path.append(os.path.abspath("/app"))

from pyrobot.indicator import Indicators
from pyrobot.stock_frame import StockFrame
from pyrobot.td_api_client import TDAPIClient
from configs.config import client_id, redirect_uri, json_path

%matplotlib inline

In [8]:
td_client = TDAPIClient(client_id, redirect_uri, json_path)

# data = td_client.get_price_histories(['goog', 'aapl'])
data = td_client.get_price_histories(['tsla', 'goog'])

In [9]:
stock_frame = StockFrame(data)
stock_frame.frame.head()

open    high     low   close    volume
symbol datetime                                                     
tsla   2019-12-16 06:00:00  72.510  76.722  72.500  76.300  91104700
       2019-12-17 06:00:00  75.798  77.100  75.180  75.798  42518875
       2019-12-18 06:00:00  76.126  79.044  76.116  78.630  70681960
       2019-12-19 06:00:00  79.464  81.370  79.300  80.808  90601535
       2019-12-20 06:00:00  82.058  82.600  80.037  81.118  73926030

In [14]:
indicator_client = Indicators(price_data_frame = stock_frame)

# 50 day sma
indicator_client.sma(period=50)

# Add the 200 day simple moving average.
indicator_client.sma(period=200)

# add daily % change in price
indicator_client.percent_change_in_price()

# # Add the 50 day exponentials moving average.
# indicator_client.ema(period=50) # TODO - fix this, window values don't make sense

# # Add the RSI Indicator.
# indicator_client.rsi(period=14)
df = indicator_client.price_data_frame
df.head()

open    high     low   close    volume  sma_50  \
symbol datetime                                                                
tsla   2019-12-16 06:00:00  72.510  76.722  72.500  76.300  91104700     NaN   
       2019-12-17 06:00:00  75.798  77.100  75.180  75.798  42518875     NaN   
       2019-12-18 06:00:00  76.126  79.044  76.116  78.630  70681960     NaN   
       2019-12-19 06:00:00  79.464  81.370  79.300  80.808  90601535     NaN   
       2019-12-20 06:00:00  82.058  82.600  80.037  81.118  73926030     NaN   

                            sma_200  %_change_in_price  Criteria 1  Criteria 2  
symbol datetime                                                                 
tsla   2019-12-16 06:00:00      NaN                NaN       False       False  
       2019-12-17 06:00:00      NaN          -0.006579       False       False  
       2019-12-18 06:00:00      NaN           0.037362       False       False  
       2019-12-19 06:00:00      NaN           0.027699       False       False  
       2019-12-20 06:00:00      NaN           0.003836       False       False

In [13]:
'''
From here on out - referencing this video:
https://www.youtube.com/watch?v=Dri1m8y9tMc
'''


# Models' buy/sell criteria
# TODO: apply this to df groups, so symbols don't overlap
df['Criteria 1'] = df['close'] >= df['sma_200']
df['Criteria 2'] = (df['sma_50'] >= df['sma_200']) | df['Criteria 1'] == True

df.tail(15)

open      high        low    close   volume  \
symbol datetime                                                                
goog   2020-11-24 06:00:00  1730.5000  1771.600  1727.6900  1768.88  1579703   
       2020-11-25 06:00:00  1772.8900  1778.540  1756.5400  1771.43  1046122   
       2020-11-27 06:00:00  1773.0900  1804.000  1772.4400  1793.19   884916   
       2020-11-30 06:00:00  1781.1844  1788.065  1755.0000  1760.74  1823836   
       2020-12-01 06:00:00  1774.3700  1824.830  1769.3700  1798.10  1739045   
       2020-12-02 06:00:00  1798.1000  1835.650  1789.4725  1827.95  1222032   
       2020-12-03 06:00:00  1824.0100  1847.200  1822.6500  1826.77  1227298   
       2020-12-04 06:00:00  1824.5200  1833.160  1816.9900  1827.99  1378166   
       2020-12-07 06:00:00  1819.0000  1832.370  1805.7800  1819.48  1320946   
       2020-12-08 06:00:00  1810.1000  1821.900  1796.2047  1818.55  1096319   
       2020-12-09 06:00:00  1812.0100  1834.270  1767.8100  1784.13  1507558   
       2020-12-10 06:00:00  1769.8000  1781.310  1740.3200  1775.33  1362848   
       2020-12-11 06:00:00  1763.0600  1784.450  1760.0000  1781.77  1220651   
       2020-12-14 06:00:00  1775.0000  1797.390  1757.2150  1760.06  1600173   
       2020-12-15 06:00:00  1764.4200  1771.420  1749.9500  1767.77  1482261   

                               sma_50      sma_200  %_change_in_price  \
symbol datetime                                                         
goog   2020-11-24 06:00:00  1594.7356  1449.778925           0.019610   
       2020-11-25 06:00:00  1599.7462  1451.044725           0.001442   
       2020-11-27 06:00:00  1605.6994  1452.437375           0.012284   
       2020-11-30 06:00:00  1611.7144  1453.637375          -0.018096   
       2020-12-01 06:00:00  1619.0532  1455.029525           0.021218   
       2020-12-02 06:00:00  1626.3030  1456.535825           0.016601   
       2020-12-03 06:00:00  1634.5342  1458.078925          -0.000646   
       2020-12-04 06:00:00  1642.5282  1459.793325           0.000668   
       2020-12-07 06:00:00  1650.0186  1461.782775          -0.004655   
       2020-12-08 06:00:00  1657.0992  1463.933275          -0.000511   
       2020-12-09 06:00:00  1663.3952  1465.888025          -0.018927   
       2020-12-10 06:00:00  1669.5098  1468.174225          -0.004932   
       2020-12-11 06:00:00  1675.3434  1470.386425           0.003627   
       2020-12-14 06:00:00  1681.3762  1472.241175          -0.012185   
       2020-12-15 06:00:00  1687.0112  1474.373075           0.004381   

                            Criteria 1  Criteria 2  
symbol datetime                                     
goog   2020-11-24 06:00:00        True        True  
       2020-11-25 06:00:00        True        True  
       2020-11-27 06:00:00        True        True  
       2020-11-30 06:00:00        True        True  
       2020-12-01 06:00:00        True        True  
       2020-12-02 06:00:00        True        True  
       2020-12-03 06:00:00        True        True  
       2020-12-04 06:00:00        True        True  
       2020-12-07 06:00:00        True        True  
       2020-12-08 06:00:00        True        True  
       2020-12-09 06:00:00        True        True  
       2020-12-10 06:00:00        True        True  
       2020-12-11 06:00:00        True        True  
       2020-12-14 06:00:00        True        True  
       2020-12-15 06:00:00        True        True

In [36]:
# Calculate models holdings over time, with 100 as the starting point
df['buy and hold'] = 100 * (1 + df['close_%_change']).cumprod()
df['200_sma'] = 100 * (1 + df['Criteria 1'].shift(1) * df['close_%_change']).cumprod()
df['200_sma_crossover'] = 100 * (1 + df['Criteria 2'].shift(1) * df['close_%_change']).cumprod()
df.tail(25)

open      high       low   close    volume  \
symbol datetime                                                              
tsla   2020-11-09 06:00:00  439.5000  452.5000  421.0000  421.26  34833025   
       2020-11-10 06:00:00  420.0900  420.0900  396.0301  410.36  30284224   
       2020-11-11 06:00:00  416.4500  418.6950  410.5800  417.13  17357722   
       2020-11-12 06:00:00  415.0500  423.0000  409.5200  411.76  19940500   
       2020-11-13 06:00:00  410.8500  412.5319  401.6600  408.50  19830351   
       2020-11-16 06:00:00  408.9300  412.4500  404.0868  408.09  26838635   
       2020-11-17 06:00:00  460.1700  462.0000  433.0100  441.61  61188281   
       2020-11-18 06:00:00  448.3500  496.0000  443.5001  486.64  78044024   
       2020-11-19 06:00:00  492.0000  508.6112  487.5700  499.27  62475346   
       2020-11-20 06:00:00  497.9900  502.5000  489.0600  489.61  32911922   
       2020-11-23 06:00:00  503.5000  526.0000  501.7900  521.85  50260304   
       2020-11-24 06:00:00  540.4000  559.9900  526.2000  555.38  53648494   
       2020-11-25 06:00:00  550.0600  574.0000  545.3700  574.00  48930162   
       2020-11-27 06:00:00  581.1600  598.7800  578.4500  585.76  37561078   
       2020-11-30 06:00:00  602.2100  607.8000  554.5100  567.60  63003052   
       2020-12-01 06:00:00  597.5900  597.8500  572.0500  584.76  40382832   
       2020-12-02 06:00:00  556.4400  571.5400  541.2100  568.82  47775653   
       2020-12-03 06:00:00  590.0200  598.9700  582.4300  593.38  42552003   
       2020-12-04 06:00:00  591.0100  599.0400  585.5000  599.04  29401314   
       2020-12-07 06:00:00  604.9197  648.7856  603.0500  641.76  56309709   
       2020-12-08 06:00:00  625.5050  651.2800  618.5000  649.88  64265029   
       2020-12-09 06:00:00  653.6900  654.3200  588.0000  604.48  71291190   
       2020-12-10 06:00:00  574.3700  627.7500  566.3400  627.07  67083153   
       2020-12-11 06:00:00  615.0100  624.0000  596.8000  609.99  46474974   
       2020-12-14 06:00:00  619.0000  642.7499  610.2001  639.83  52040649   

                              sma_50    sma_200  close_%_change  Criteria 1  \
symbol datetime                                                               
tsla   2020-11-09 06:00:00  422.3004  254.02587       -0.020212        True   
       2020-11-10 06:00:00  420.5412  255.51077       -0.025875        True   
       2020-11-11 06:00:00  419.3828  257.01543        0.016498        True   
       2020-11-12 06:00:00  418.6706  258.43342       -0.012874        True   
       2020-11-13 06:00:00  418.7006  259.82535       -0.007917        True   
       2020-11-16 06:00:00  418.4960  261.08580       -0.001004        True   
       2020-11-17 06:00:00  420.7240  262.40679        0.082139        True   
       2020-11-18 06:00:00  423.1312  264.10529        0.101968        True   
       2020-11-19 06:00:00  425.6898  265.85268        0.025953        True   
       2020-11-20 06:00:00  428.0276  267.55266       -0.019348        True   
       2020-11-23 06:00:00  430.0722  269.39063        0.065848        True   
       2020-11-24 06:00:00  432.1846  271.39315        0.064252        True   
       2020-11-25 06:00:00  434.8294  273.49586        0.033527        True   
       2020-11-27 06:00:00  438.0760  275.62066        0.020488        True   
       2020-11-30 06:00:00  440.5850  277.65863       -0.031002        True   
       2020-12-01 06:00:00  443.2924  279.72403        0.030233        True   
       2020-12-02 06:00:00  446.1842  281.65071       -0.027259        True   
       2020-12-03 06:00:00  450.4446  283.71820        0.043177        True   
       2020-12-04 06:00:00  454.6696  285.81240        0.009539        True   
       2020-12-07 06:00:00  459.3580  288.18741        0.071314        True   
       2020-12-08 06:00:00  463.9316  290.63690        0.012653        True   
       2020-12-09 06:00:00  467.6398  292.88050       -0.069859        True   
       2020-12-10 06:00:00  4

In [37]:
# Calculate Model Returns

# TODO: adapt this to work for df groups

# 200 sma model
start_model1 = df['200_sma'].iloc[200]
end_model1 = df['200_sma'].iloc[-1]
years = (df['200_sma'].count() + 1 - 200)/252
model1_avg_return = (end_model1/start_model1)**(1/years) - 1
print('200 sma model yeilds an average of ', model1_avg_return * 100, '% per year')


# 200 sma crossover model
start_model2 = df['200_sma_crossover'].iloc[200]
end_model2 = df['200_sma_crossover'].iloc[-1]
model2_avg_return = (end_model2/start_model2)**(1/years) - 1
print('200 sma model yeilds an average of ', model2_avg_return * 100, '% per year')


# buy and hold
start = df['close'].iloc[200]
end = df['close'].iloc[-1]
avg_return = (end/start)**(1/years) - 1
print('Buy and hold yeilds an average of ', avg_return * 100, '% per year')

200 sma model yeilds an average of  461.5236802776399 % per year
200 sma model yeilds an average of  461.5236802776399 % per year
Buy and hold yeilds an average of  461.5236802776399 % per year
